# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')

In [3]:
# 결측치를 처리하기 전에,
# 결측 여부가 failure를 예측하는데, 유용할 만하다고 도출된
# measurement_3, measurement_5의 결측 여부만 남깁니다.
df_train[['na_1', 'na_2']] = df_train[['measurement_3', 'measurement_5']].isna()
df_test[['na_1', 'na_2']] = df_test[['measurement_3', 'measurement_5']].isna()

In [4]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [5]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [6]:
from sklearn.experimental import enable_iterative_imputer# 구문을 사용하여 실험 단계인 모듈을 활성화하고, 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
# train에 등장하지 않은 수준이 있습니다, test를 포함하여 결측처리 모델을 만듭니다.
s_imp = pd.concat([
                df_train[X_imp + ['product_code']],
                df_test[X_imp + ['product_code']]
        ], axis=0).groupby('product_code')\
        .apply(
            lambda x: IterativeImputer(estimator=LinearRegression(),random_state=123).fit(x[X_imp])
        )
# train에 적용합니다.
df_train[X_imp] = df_train[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )
# test에 적용합니다.
df_test[X_imp] = df_test[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )

In [7]:
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
# 역시 train에 등장하지 않은 수준을 처리하기 위해 합치니다.
df_mean = pd.concat([
            df_train[['product_code'] + X_mean],
            df_test[['product_code'] + X_mean]
        ]).groupby('product_code')[X_mean].agg('mean')

df_train[X_mean] = df_train.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))
df_test[X_mean] = df_test.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))

In [8]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
# loading은 train에서의 평균으로 결측치를 처리합니다.
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())
df_train.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0)

In [9]:
# 공통으로 사용할 만한 요소입니다.
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True) # 5겹의 층화교차검증을 사용합니다.

In [12]:
# Baseline 모델을 만들어봅니다.
from sklearn.linear_model import LogisticRegression
# loading, measurment_0~18까지 표준화합니다.
# na_1, na_2는 통과시킵니다.
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading'] + ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['na_1', 'na_2'])
])

X_lr = ['loading'] + ['measurement_{}'.format(i) for i in range(18)] + ['na_1', 'na_2']
clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
# Baseline 모델의 성능을 평가합니다.
scores = cross_val_score(clf_lr, df_train[X_lr], df_train['failure'], scoring='roc_auc', cv=cv)
scores, np.mean(scores)

(array([0.57912402, 0.58968448, 0.57221316, 0.61180378, 0.57922945]),
 0.5864109791862437)

In [13]:
# 제출을 위한 모델을 만듭니다.
clf_lr.fit(df_train[X_lr], df_train['failure'])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('std',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['loading', 'measurement_0',
                                                   'measurement_1',
                                                   'measurement_2',
                                                   'measurement_3',
                                                   'measurement_4',
                                                   'measurement_5',
                                          

In [14]:
# 제출 파일을 만듭니다.
pd.DataFrame(
    clf_lr.predict_proba(df_test[X_lr])[:, 1], # failure=1인 확률을 뽑아냅니다.
    index=df_test.index
).to_csv('answer6.csv')

In [16]:
# 채점을 해봅니다.
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')

In [17]:
roc_auc_score(df_ans['failure'], clf_lr.predict_proba(df_test[X_lr])[:, 1])

0.5891265737410073

In [18]:
# 문제에서 속성 선택법에 대한 힌트가 없을 때를 가정하고 
# 빠르게 속성선택법의 효과성을 살펴 봅니다.

# 속성선택기 설정을 합니다.
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True) # 5겹 층화교차검증(5-Fold stratified cross validation)
sfs = SequentialFeatureSelector(
    estimator = LogisticRegression(solver='lbfgs'),
    k_features='best',# 최적의 성능을 보이는 입력 변수의 조합을 찾는다. 
    forward=True, # 전진 선택
    floating=False, # 선택했던 변수를 제외하지 않는다. 
    cv = cv,
    scoring='roc_auc',
    n_jobs=4 # 좀 더 빨리 뽑아오기 위해 동시 실행을 4로 잡습니다.
)

c:\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
from sklearn.linear_model import LogisticRegression

ct = ColumnTransformer([
    ('std', StandardScaler(),  ['loading'] + ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['na_1', 'na_2'])
])
X_lr = ['loading'] + ['measurement_{}'.format(i) for i in range(18)] + ['na_1', 'na_2']
# 속성 선택기를 pipeline 중간에 넣습니다.
clf_lr = make_pipeline(ct, sfs, LogisticRegression(solver='lbfgs'))
scores = cross_val_score(clf_lr, df_train[X_lr], df_train['failure'], cv=cv, scoring='roc_auc')
scores, np.mean(scores)

(array([0.57899225, 0.59057654, 0.5762483 , 0.60776799, 0.58230052]),
 0.5871771208338508)

In [20]:
# 모든 train 셋으로 학습시킵니다.
clf_lr.fit(df_train[X_lr], df_train['failure'])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('std',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['loading', 'measurement_0',
                                                   'measurement_1',
                                                   'measurement_2',
                                                   'measurement_3',
                                                   'measurement_4',
                                                   'measurement_5',
                                          

In [21]:
# 속성 선택기가 도출한 입력을 봅니다, ColumnTransformer를 거쳐 변수명이 추상화됐습니다.
clf_lr[1].k_feature_names_

('0', '2', '5', '10', '15', '18', '19')

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# ColumnTransformer에 PCA 파이프라인들 구상했습니다.
# 이러한 중첩된 구성 또한 성립합니다.
ct = ColumnTransformer([
    ('pca', make_pipeline(StandardScaler(), PCA(n_components=7)), ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough',  ['loading'] + ['na_1', 'na_2'])
])
# GridSearchCV 모델로 구성해봅니다.
gscv = GridSearchCV(
    estimator=RandomForestClassifier(random_state=123),
    param_grid={
        'n_estimators': [5, 10, 15],
        'max_depth': [5, 6, 7],
        'min_samples_split': [256, 512]
    },
    scoring='roc_auc',
    iid=False,
    cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
    n_jobs=4
)
X_gscv = ['loading'] + ['measurement_{}'.format(i) for i in range(18)] + ['na_1', 'na_2']
clf_gscv_rf = make_pipeline(ct, gscv)
scores_ = cross_val_score(clf_gscv_rf, df_train[X_gscv], df_train['failure'], scoring='roc_auc', cv=cv)
scores_, np.mean(scores_)

(array([0.58366693, 0.59268903, 0.5654862 , 0.59334217, 0.57527332]),
 0.5820915301430556)

In [23]:
# 전체 데이터셋에 대한 학습을 합니다.
clf_gscv_rf.fit(df_train[X_gscv], df_train['failure'])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pca',
                                                  Pipeline(memory=None,
                                                           steps=[('standardscaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('pca',
                                                                   PCA(copy=True,
                                                                       iterated_power='auto',
  

In [24]:
# 결과 hyper-parameter를 봅니다.
clf_gscv_rf[1].best_params_

{'max_depth': 6, 'min_samples_split': 512, 'n_estimators': 15}

# 다른 모델들도 만들어 봅니다. 

참고삼아 만들어 봣습니다.
그리고, 이들을 마지막 앙상블 모델에 넣어 보겟습니다.

In [25]:
# GradientBoosting 모델을 만들어 봅니다.
from sklearn.ensemble import GradientBoostingClassifier
X_gb = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct_3 = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])
clf_gb = make_pipeline(ct_3, GradientBoostingClassifier(
    n_estimators=100, max_depth=2, learning_rate=0.01, random_state=123
))
scores_ = cross_val_score(clf_gb, df_train[X_gb], df_train['failure'], cv=cv, scoring='roc_auc')
scores_, np.mean(scores_)

(array([0.59016944, 0.59366441, 0.57107023, 0.59631159, 0.57999249]),
 0.586241632091449)

In [26]:
# RandomForestClassifier 모델도 만들어 봅니다. (좀더 튜닝했습니다.)
from sklearn.ensemble import RandomForestClassifier
X_rf = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct_4 = ColumnTransformer([
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=7)), ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['loading', 'na_1', 'na_2'])
])
clf_rf = make_pipeline(ct_4, RandomForestClassifier(
    n_estimators=100, max_depth=7, min_samples_split= 512, random_state=123
))
scores_ = cross_val_score(clf_rf, df_train[X_rf], df_train['failure'], cv=cv, scoring='roc_auc')
scores_, np.mean(scores_)

(array([0.58362268, 0.5903637 , 0.56514573, 0.59496567, 0.57666012]),
 0.5821515804775613)

In [27]:
# xgb로 모델을 만들어 봅니다.
import xgboost as xgb
X_xgb = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct_5 = ColumnTransformer([
    ('pt', 'passthrough', X_xgb)
])
clf_xgb = make_pipeline(ct_3, xgb.XGBClassifier(
    n_estimators=150, max_depth=2, learning_rate=0.01, random_state=123
))
scores_ = cross_val_score(clf_xgb, df_train[X_xgb], df_train['failure'], cv=cv, scoring='roc_auc')
scores_, np.mean(scores_)

(array([0.58961081, 0.59397   , 0.5726996 , 0.59593267, 0.57998353]),
 0.5864393237715635)

In [28]:
# Baseline입니다.
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading'] + ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['na_1', 'na_2'])
])
X_lr = ['loading'] + ['measurement_{}'.format(i) for i in range(18)] + ['na_1', 'na_2']
clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))

# 속성 선택 + LogisticRegression 모델입니다.
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
X_lr = ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']
clf_lr_2 = make_pipeline(ct, LogisticRegression(solver='lbfgs'))

# PCA + LogisticRegression 모델입니다.
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading']),
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=7)) , ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['na_1', 'na_2'])
])
X_lr = ['loading'] + ['measurement_{}'.format(i) for i in range(18)] + ['na_1', 'na_2']
clf_lr_3 = make_pipeline(ct, LogisticRegression(solver='lbfgs'))

# GradientBoosting 모델입니다.
X_gb = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])
clf_gb = make_pipeline(ct, GradientBoostingClassifier(
    n_estimators=100, max_depth=2, learning_rate=0.01, random_state=123
))

# Random Forest 입니다.
X_rf = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct = ColumnTransformer([
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=7)), ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['loading', 'na_1', 'na_2'])
])
clf_rf = make_pipeline(ct, RandomForestClassifier(
    n_estimators=100, max_depth=7, min_samples_split= 512, random_state=123
))

# XGBoost 입니다.
X_xgb = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_xgb)
])
clf_xgb = make_pipeline(ct, xgb.XGBClassifier(
    n_estimators=150, max_depth=2, learning_rate=0.01, random_state=123
))

In [29]:
from sklearn.ensemble import VotingClassifier
X_vt = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)] 
# 모두 앙상블을 해봅니다. roc_auc에서 확률을 활용하므로 voting 을 soft로 합니다.
clf_vt = VotingClassifier(
    [
        ('lr', clf_lr), #  Baseline
        ('lr_2', clf_lr_2), # 속성 선택을 통한 모델
        ('lr_3', clf_lr_3), # PCA + 속성 선택 모델 
        ('gb', clf_gb), # GradientBoost
        ('rf', clf_rf), # Random Forest
        ('xgb', clf_xgb) # xgboost
    ],
    voting='soft'
)
scores_ = cross_val_score(clf_vt, df_train[X_vt], df_train['failure'], cv=cv, scoring='roc_auc')
score = np.mean(scores_)
scores_, score

(array([0.58740394, 0.59446078, 0.57334307, 0.60939866, 0.58233571]),
 0.5893884319103216)

In [30]:
# 전체 train에 대해 만들고, 채점도 해봅니다.
clf_vt.fit(df_train[X_lr], df_train['failure'])
roc_auc_score(df_ans['failure'], clf_vt.predict_proba(df_test[X_lr])[:, 1])

0.5928660071942446